In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

MODEL = "llama2"

In [2]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings()
model.invoke("tell me a joke")

"Sure! Here's one:\n\nWhy don't scientists trust atoms?\nBecause they make up everything!\n\nI hope you found that amusing! Do you want to hear another one?"

In [3]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("DAA.pdf")
pages = loader.load_and_split()
pages

[Document(page_content='www.pes.edu \n \n \n \n \n \n \n \nDivide and Conquer Approach \nBinary Search \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \nDr. Shylaja S S Department of Computer Science and Engineering \nPES UNIVERSITY \nUE19CS251 : Design and Analysis of Algorithms (4-0-0-4- 4)', metadata={'source': 'DAA.pdf', 'page': 0}),
 Document(page_content="www.pes.edu \nDivide-and-C onquer Approach \nDivide-and-Conquer is probably the best-known general algorithm design  \ntechnique. Though its fame may have something to do with its catchy name, it \nis well deserved: quite a few very efficient algorithms are specific \nimplementations of this general strategy.   Divide-and-conquer algorithms work \naccording to the following general plan:  \n1.  A  problem's  instance  is  divided  into  several  smaller  instanc es  of the  \nsame problem, ideally of about the same size.  \n2.  The smaller instances are solved (typically recursively, though sometimes a \ndifferent algorithm is e

In [4]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't answer the question, reply "I dont know".

Context: {context}
Question: {question}
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(question="this is a question", context="this is a context"))


Answer the question based on the context below. If you can't answer the question, reply "I dont know".

Context: this is a context
Question: this is a question



In [5]:
chain = prompt | model

In [6]:
chain.invoke({
    "question": "What is my name",
    "context": "the name i was given is samarth"
})

'Sure! Based on the context provided, your name is Samarth.'

In [7]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)


In [8]:
retriever = vectorstore.as_retriever()
retriever.invoke("what is master theorem?")

[Document(page_content='Applications:  \n1. Most of the self-balancing BST library functions like map and  set in \nC++ (OR TreeSet and TreeMap in Java) use Red-Black Tree. \n2. It is used to implement CPU Scheduling Linux. Completely Fai r \nScheduler uses it. \n3. Besides they are used in the K-mean clustering algorithm for reducing \ntime complexity. \n4. Moreover, MySQL also uses the Red-Black tree for indexes on tab les. \n \nInsertion in Red Black Tree: \nAlgorithm: \nLet x be the newly inserted node. \n1. Perform standard BST insertion and make the colour of newly i nserted \nnodes as RED. \n2. If x is the root, change the colour of x as BLACK (Black h eight of \ncomplete tree increases by 1). \n3. Do the following if the color of x’s parent is not BLACK  and x is not \nthe root.  \na) If x’s uncle is RED  (Grandparent must have been black \nfrom property  \n4). (i)  Change the colour of parent and uncle as BLACK.', metadata={'source': 'DAA.pdf', 'page': 28}),
 Document(page_con

In [9]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
)


In [10]:
questions = [
    "What is the master theorem?",
    "what is quick sort?",
    "what is merge sort?",
    "what is a red black tree?"
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

Question: What is the master theorem?
Answer: Based on the provided context, the Master Theorem is a data structure used in Red-Black Tree insertion algorithm. It is a way of categorizing the insertion operations based on the color of the tree and the position of the newly inserted node. The Master Theorem has three cases, which depend on the color of the grandparent of the newly inserted node. These cases are:

1. Left Left Case: If the grandparent is RED and the newly inserted node is left child of its parent, then the algorithm changes the colors of the grandparent, parent, and newly inserted node to BLACK, RED, and BLACK, respectively.
2. Left Right Case: If the grandparent is RED and the newly inserted node is right child of its parent, then the algorithm changes the colors of the grandparent, parent, and newly inserted node to BLACK, RED, and BLACK, respectively.
3. Right Right Case: If the grandparent is BLACK, then there are four possible configurations for the newly inserted n